In [ ]:
import numpy as np
import visualization.plots 
import datasets.control

In [ ]:
from datasets.control import get_mixture_distribution

mixture_list = [
    ("uniform", [-10, -5], [5, 10]),
    ("normal", [10, 5], [2, 12]),
    ("normal", 0, 1),
    ("normal", [3, 6], [4, 5]),
    ("laplace", 4, 1),
    ("normal", -5, 3),
    ("exponential", 1, -5),  # shift to center around -5
]

weights = [0.1, 0.2, 0.2, 0.1, 0.1, 0.2, 0.1]

data = get_mixture_distribution(mixture_list, weights, size=(100000, 2))

In [ ]:
visualization.plots.plot_3d_kde(data)